In [1]:
# !git clone https://github.com/fadhleryani/malti_arabi_fst.git
# # !git pull

# %pip install pynini
# %pip install pyfoma

In [2]:
# %cd malti_arabi_fst

In [5]:
import pynini as pn
import kenlm
from itertools import product
import pyconll
import pandas as pd
import numpy as np
from transformers import AutoTokenizer

/opt/homebrew/Caskroom/miniconda/base/envs/maltifst/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix")

In [7]:
wordmodel = kenlm.Model('data/arabi_data/arabic_lm/aggregated_country/lm/word/tn-maghreb.arpa')
charmodel = kenlm.Model('data/arabi_data/arabic_lm/aggregated_country/lm/char/tn-maghreb.arpa')

Loading the LM will be faster if you build a binary file.
Reading /Users/f/ba3sasah/malti_arabi_fst/data/arabi_data/arabic_lm/aggregated_country/lm/word/tn-maghreb.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.
Reading /Users/f/ba3sasah/malti_arabi_fst/data/arabi_data/arabic_lm/aggregated_country/lm/char/tn-maghreb.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [8]:
def merge_conllu(dataset,charhist=False):
    dev = pyconll.load_from_file(f'data/malti_data/{dataset}/dev.conllu')
    train = pyconll.load_from_file(f'data/malti_data/{dataset}/train.conllu')
    test = pyconll.load_from_file(f'data/malti_data/{dataset}/test.conllu')
    allsets = dev._sentences + train._sentences + test._sentences

    print(f'# of sents in {dataset}',len(allsets))

    keys = ["id","form","lemma","upos","xpos","feats","head","deprel","deps","misc"]

    sents = []
    for sent in allsets:
        # toks = [pd.Series({'sent_id':sent.id,'sent':sent.text})]
        toks = []
        for tok in sent:
            tokdict = {'sent_id':sent.id}
            tokdict.update( {k:tok.__getattribute__(k) for k in keys})
            toks.append (pd.Series(tokdict))
        sents.append(pd.DataFrame(toks))
    df = pd.concat(sents)    
    # word_hist
    word_hist = df['form'].dropna().value_counts().reset_index()
    # word_hist.to_clipboard()
    print(f'# of words in {dataset}',len(word_hist))
    # # char hist
    char_hist = pd.DataFrame([y for x in df['form'].dropna().str.casefold() for y in x]).value_counts()
    # char_hist.to_clipboard()
    print(f'# of chars {dataset}',len(char_hist))
    return word_hist

In [9]:
mlrs = merge_conllu('MLRS POS')
sa = merge_conllu('Sentiment Analysis')
mapa = merge_conllu('MAPA')
mudt = merge_conllu('mt_mudt-ud')

# of sents in MLRS POS 6167
# of words in MLRS POS 15774
# of chars MLRS POS 81
# of sents in Sentiment Analysis 851
# of words in Sentiment Analysis 5425
# of chars Sentiment Analysis 70
# of sents in MAPA 8763
# of words in MAPA 19059
# of chars MAPA 143
# of sents in mt_mudt-ud 2074
# of words in mt_mudt-ud 8471
# of chars mt_mudt-ud 74


In [6]:
# closedclass = pd.read_csv('mappings/closed_class_mappings.tsv',sep='\t',header=None) # already unique 
# closedclass = dict(closedclass.values)



In [61]:

malti2arabi_2char = pn.string_file('mappings/malti2arabi_2char.map').optimize()
arabic2arabic = pn.string_file('mappings/arabic2arabic.map').optimize()
malti2arabi_1char = pn.string_file('mappings/malti2arabi_1char.map').optimize()
shadda = pn.string_file('mappings/shadda.map').optimize()
final_vowels = pn.string_file('mappings/final_vowels.map').optimize()
special = pn.string_file('mappings/special.map').optimize()
alif_initial = pn.string_file('mappings/alif_initial.map').optimize()
baby_closed_class = pn.string_file('mappings/baby_closed_class.map').optimize()

sigma_malti = pn.project(malti2arabi_1char,'input')
sigma_arabi = pn.project(arabic2arabic,'output') 

# SIGMA
sigma_in = pn.project(pn.union(malti2arabi_1char,special,arabic2arabic,final_vowels),'input')
sigma = pn.project(pn.union(sigma_in,special,final_vowels),'output')
sigma = pn.union(sigma,"-").optimize() 

rwr_first_fsts = pn.union(
    malti2arabi_2char,
    shadda,
    final_vowels,
    alif_initial,
).optimize()

rwr_first = pn.cdrewrite(rwr_first_fsts,"","",sigma.closure())

second_fsts = pn.union(
    malti2arabi_1char,
    arabic2arabic, 
    special,
).optimize()

translit_fst = rwr_first @ second_fsts.closure()

malti2arabi_det= pn.string_file('mappings_deterministic/malti2arabi_1char_vowels_short.map').optimize()
special_deterministic = pn.string_file('mappings_deterministic/special_deterministic.map').optimize()

diacs = 'ًٌٍَُِّْ'
dediac_cross = pn.string_file('mappings/dediac.map')
dediac = pn.cdrewrite(dediac_cross,'','',sigma.closure())

augmented_closed_class = pn.string_file('mappings/augmented_closed_class.map').optimize()

words = pn.string_file('data/arabi_data/tn-maghreb-words.txt').optimize() @ dediac

In [8]:
# paths = get_paths(("<BOS>mil-<EOS>"  @ baby_closed_class @ transcriber  )),get_paths(("<BOS>fok<EOS>"   @ transcriber  ))
# paths,len(paths)

In [12]:
def dediac_fst(text):
    text = text.replace('[','\[').replace(']','\]')
    try:
        return (text @ dediac).string()
    except:
        return np.nan
    
words_df= pd.read_fwf('data/arabi_data/tn-maghreb-words.txt',header=None).rename(columns={0:'words'})
words_df['dediac'] = pd.Series([dediac_fst(x) for x in words_df['words']])

# words_df

In [76]:
def get_paths(fst,words_only=False):
    paths = list(fst.paths().items())
    if words_only:
        return [x[1] for x in paths]
    else:
        return paths



def apply_translit_fst(tok,backoff_fsts=[baby_closed_class,augmented_closed_class]):
    tok = tok.replace('[','\[').replace(']','\]')
    tok = (f'<BOS>{tok}<EOS>')
    # if type=='det':
    #     return tok @ deterministic_transcriber @ dediac
    if backoff_fsts:
        backoff =  tok @ pn.union(*backoff_fsts).optimize() @ dediac
        if get_paths(backoff):
            return backoff
        else:
            return tok  @ translit_fst @ dediac
    else:
        return tok  @ translit_fst @ dediac

def filter_edge_diacritics(options):
    return [y for y in options if y[0] not in diacs and y[-1] not in diacs]

def translit_deterministic(lowered,backoffs=[]):
    lowered = lowered.replace('[','\[').replace(']','\]') 
    if backoffs:
        backofflowered = f'<BOS>{lowered}<EOS>'
        backoff = backofflowered @ pn.union(*backoffs).optimize() @ dediac
        if len(get_paths(backoff))==1:
            return backoff.string()
        elif len(get_paths(backoff))>1:
            print('error: fst is NOT deterministic on:',lowered)
            return '#na'
    # default
    try:
        maptranslit = (lowered @ pn.union(malti2arabi_det,special_deterministic).closure().optimize() @ dediac).string()
        return maptranslit
    except:
        print('error detfst on:',lowered)
        return '#na'
            

def translit_word(lowered_tok,backoffs): #select on merged but return unmerged

    tok_fst = apply_translit_fst(lowered_tok,backoffs)
    translit_toks = get_paths(tok_fst,words_only=True) 
    if not translit_toks:
        return ['#NA']
    try:
        translit_toks = filter_edge_diacritics(translit_toks) 
    except:
        print('err filtering diacs',translit_toks,lowered_tok)
    
    translit_toks = [ dediac_fst(x) for x in translit_toks]  # dediacritize
    return translit_toks
    

langmodelset =  set(words_df['dediac'])

def count_subtokens(text, tokenizer):
    return tokenizer(text, add_special_tokens=False, return_length=True)["length"]


def translit_and_rank_options(word,name='translit',fsttype='non-det',backoffs=[baby_closed_class,augmented_closed_class]):
  
    lowered = word.lower()
    translit_dict = {
        'word_raw':word,
        'word_lowered':lowered,
        }
    
    if fsttype == 'det':
        translit = [translit_deterministic(lowered)]
    elif fsttype == 'non-det':
        translit = translit_word(lowered,backoffs)
    else:
        raise Exception('wrong fsttype')

    translit_dict[name] = translit
    translit_dict['translit'] = translit # keep this, in order to merge later
    translit_dict['translit_stripped'] = [x.rstrip('+') for x in translit]
    translit_dict['wordmodel_score'] = [wordmodel.score(x) for x in translit_dict['translit_stripped']]
    translit_dict['charmodel_score'] = [charmodel.score(' '.join(x)) for x in translit_dict['translit_stripped'] ]
    translit_dict['capitalized'] = word[0].isupper() # TODO: what about letter after sink as in 'L-Innu', does it matter?
    translit_dict['in_langmodel'] = [x in langmodelset for x in translit_dict['translit_stripped']]
    translit_dict['subtokens'] = count_subtokens(translit_dict['translit_stripped'], tokenizer)
    translit_dict['subtokens_lowest_ties'] = sum(np.array(translit_dict['subtokens']) == min(translit_dict['subtokens']))

    return translit_dict
    

word = "t'"
word = "L-Innu"
word = "din"
word = "f'dik"
word = "d-dinja"
word = "f'dil-konferenza d-dinja"
word = "mil-dinja" # check how many tokens it breaks into and how that affects lang model scores
word = "id-"
word = "fil- linja ."
word = "m'" 
word = "a"
word = "uffiċjali"
word = "il-"
word = "tielgħa"
# word = "[għandhomx" 
# translit_and_rank_options(word,cutoff=None,useclosedclass=False).merge(translit_and_rank_options(word,cutoff=None,useclosedclass=True),how='outer').sort_values(['charmodel_score'],ascending=False)
# translit_and_rank_options(word,cutoff=None,useclosedclass=False).merge(translit_and_rank_options(word,cutoff=None,useclosedclass=True),how='outer').sort_values(['wordmodel_score'],ascending=False)
# sorted([(wordmodel.score(x),x) for x in merged],key=lambda x: -x[0])



def generate_table(word):
    det = translit_and_rank_options(word,name='det',fsttype='det')    
    det_smallcc = translit_and_rank_options(word,name='det_smallcc',fsttype='det', backoffs=[baby_closed_class])    
    det_fullcc = translit_and_rank_options(word,name='det_fullcc',fsttype='det', backoffs=[baby_closed_class,augmented_closed_class])    
    nondet = translit_and_rank_options(word,name='nondet',fsttype='non-det')
    nondet_smallcc = translit_and_rank_options(word,name='nondet_smallcc',fsttype='non-det',backoffs=[baby_closed_class])
    nondet_fullcc = translit_and_rank_options(word,name='nondet_fullcc',fsttype='non-det',backoffs=[baby_closed_class,augmented_closed_class])
    
    return (det,det_smallcc,det_fullcc,nondet,nondet_smallcc,nondet_fullcc,)
   
det,det_smallcc,det_fullcc,nondet,nondet_smallcc,nondet_fullcc = generate_table(word)
det = pd.DataFrame(det)
det_smallcc = pd.DataFrame(det_smallcc)
det_fullcc = pd.DataFrame(det_fullcc)
nondet = pd.DataFrame(nondet)
nondet_smallcc = pd.DataFrame(nondet_smallcc)
nondet_fullcc = pd.DataFrame(nondet_fullcc)

def merge_multiple(dfs=[det,det_smallcc,det_fullcc,nondet,nondet_smallcc,nondet_fullcc]):
    first = dfs[0]
    for df in dfs[1:]:
        first = first.merge(df,how='outer')
    
    return first.sort_values('wordmodel_score',ascending=False)[[
        'translit',
        'det',
        'det_smallcc',
        'det_fullcc',
        'nondet',
        'nondet_smallcc',
        'nondet_fullcc',        
        'translit_stripped',
        'word_raw',
        'word_lowered',
        'wordmodel_score',
        'charmodel_score',
        'capitalized',
        'in_langmodel',
        'subtokens',
        # 'subtokens_lowest_ties',
        ]]

merged = merge_multiple()
merged

,translit,det,det_smallcc,det_fullcc,nondet,nondet_smallcc,nondet_fullcc,translit_stripped,word_raw,word_lowered,wordmodel_score,charmodel_score,capitalized,in_langmodel,subtokens
1,طالعة,NaN,NaN,NaN,طالعة,NaN,طالعة,طالعة,tielgħa,tielgħa,-6.705869,-6.622174,False,True,2
11,طالعة,NaN,NaN,NaN,NaN,طالعة,NaN,طالعة,tielgħa,tielgħa,-6.705869,-6.622174,False,True,2
12,طالعه,NaN,NaN,NaN,NaN,طالعه,NaN,طالعه,tielgħa,tielgħa,-7.818769,-7.795122,False,True,2
0,تلجح,تلجح,تلجح,تلجح,NaN,NaN,NaN,تلجح,tielgħa,tielgħa,-8.091814,-9.504936,False,False,2
16,طالغه,NaN,NaN,NaN,NaN,طالغه,NaN,طالغه,tielgħa,tielgħa,-8.091814,-10.538343,False,False,2
15,طالغة,NaN,NaN,NaN,NaN,طالغة,NaN,طالغة,tielgħa,tielgħa,-8.091814,-10.012894,False,False,2
14,طالغا,NaN,NaN,NaN,NaN,طالغا,NaN,طالغا,tielgħa,tielgħa,-8.091814,-11.608114,False,False,2
13,طالعى,NaN,NaN,NaN,NaN,طالعى,NaN,طالعى,tielgħa,tielgħa,-8.091814,-9.582643,False,False,2
10,طالعا,NaN,NaN,NaN,NaN,طالعا,NaN,طالعا,tielgħa,tielgħa,-8.091814,-11.131311,False,False,2
9,تالغى,NaN,NaN,NaN,NaN,تالغى,NaN,تالغى,tielgħa,tielgħa,-8.091814,-10.442297,False,False,2


In [64]:
def translit_deterministic(lowered,backoffs=[]):
    lowered = lowered.replace('[','\[').replace(']','\]') 
    if backoffs:
        backofflowered = f'<BOS>{lowered}<EOS>'
        backoff = backofflowered @ pn.union(*backoffs).optimize() @ dediac
        if len(get_paths(backoff))==1:
            return backoff.string()
        elif len(get_paths(backoff))>1:
            print('error: fst is NOT deterministic on:',lowered)
            return '#na'
    # default
    try:
        maptranslit = (lowered @ pn.union(malti2arabi_det,special_deterministic).closure().optimize() @ dediac).string()
        return maptranslit
    except:
        print('error detfst on:',lowered)
        return '#na'
        
            
    
translit_word("14-il",backoffs=[baby_closed_class]),translit_deterministic("14-il",backoffs=[baby_closed_class])


(['14ل'], '14+ل')

In [69]:
translit_deterministic('ab')

In [77]:
# word_hist = word_hist
# mudtdev_translit = pd.concat(word_hist['sent'].iloc[:].apply(translit_and_rank_options).values)
def translit_dataset(word_hist):
    
    detlist = []
    det_smallcclist = []
    det_fullcclist = []
    nondetlist = []
    nondet_smallcclist = []
    nondet_fullcclist = []

    for word,freq in word_hist.values[:]:
        det, det_smallcc, det_fullcc, nondet, nondet_smallcc, nondet_fullcc = generate_table(word)
        det['freq'] = freq
        det_smallcc['freq'] = freq
        det_fullcc['freq'] = freq
        nondet['freq'] = freq
        nondet_smallcc['freq'] = freq
        nondet_fullcc    ['freq'] = freq
        
        detlist.append(det)
        det_smallcclist.append(det_smallcc)
        det_fullcclist.append(det_fullcc)
        nondetlist.append(nondet)
        nondet_smallcclist.append(nondet_smallcc)
        nondet_fullcclist.append(nondet_fullcc)
        
    
    detlist = pd.DataFrame(detlist).explode(['translit','detlist','translit_stripped','wordmodel_score','charmodel_score','in_langmodel','subtokens'])
    det_smallcclist = pd.DataFrame(det_smallcclist).explode(['translit','det_smallcclist','translit_stripped','wordmodel_score','charmodel_score','in_langmodel','subtokens'])
    det_fullcclist = pd.DataFrame(det_fullcclist).explode(['translit','det_fullcclist','translit_stripped','wordmodel_score','charmodel_score','in_langmodel','subtokens'])
    nondetlist = pd.DataFrame(nondetlist).explode(['translit','nondetlist','translit_stripped','wordmodel_score','charmodel_score','in_langmodel','subtokens'])
    nondet_smallcclist = pd.DataFrame(nondet_smallcclist).explode(['translit','nondet_smallcclist','translit_stripped','wordmodel_score','charmodel_score','in_langmodel','subtokens'])
    nondet_fullcclist = pd.DataFrame(nondet_fullcclist).explode(['translit','nondet_fullcclist','translit_stripped','wordmodel_score','charmodel_score','in_langmodel','subtokens'])

    return merge_multiple(dfs=
                          [
detlist,
det_smallcclist,
det_fullcclist,
nondetlist,
nondet_smallcclist,
nondet_fullcclist,
                          ]
                          )

    

In [78]:
mudt_translit = translit_dataset(mudt)
len(mudt_translit),len(mudt_translit),len(mudt_translit)/len(mudt_translit['word_raw'].unique())
mapa_translit = translit_dataset(mapa)
len(mapa_translit),len(mapa_translit),len(mapa_translit)/len(mapa_translit['word_raw'].unique())
mlrs_translit = translit_dataset(mlrs)
len(mlrs_translit),len(mlrs_translit),len(mlrs_translit)/len(mlrs_translit['word_raw'].unique())
sa_translit = translit_dataset(sa)
len(sa_translit),len(sa_translit),len(sa_translit)/len(sa_translit['word_raw'].unique())

error detfst on: ×
error detfst on: ×
error detfst on: ×
error detfst on: \rin\[
error detfst on: \rin\[
error detfst on: \rin\[


NameError: name 'detlistlist' is not defined

In [ ]:
mudt_translit.to_csv('transliterations/mudt_transliterated_tuples.tsv',sep='\t',index=False)
mapa_translit.to_csv('transliterations/mapa_transliterated_tuples.tsv',sep='\t',index=False)
mlrs_translit.to_csv('transliterations/mlrs_transliterated_tuples.tsv',sep='\t',index=False)
sa_translit.to_csv('transliterations/sa_transliterated_tuples.tsv',sep='\t',index=False)

In [20]:
sa_translit

,freq,word_raw,word_lowered,translit,deterministic,small_cc,augmented_cc,translit_stripped,wordmodel_score,charmodel_score,capitalized,in_langmodel,subtokens,subtokens_lowest_ties
0,1268,.,.,.,.,.,.,.,-3.556239,-5.553323,False,True,1,1
1,640,li,li,اللي,NaN,NaN,اللي,اللي,-4.311847,-5.119282,False,True,1,1
2,640,li,li,لي,NaN,لي,NaN,لي,-4.859493,-5.063514,False,True,1,1
3,640,li,li,ل,ل,NaN,NaN,ل,-5.295598,-4.732144,False,True,1,1
4,598,u,u,,,NaN,NaN,,-2.06162,-4.263644,False,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132012,1,000,000,000,000,000,000,000,-7.459375,-8.066975,False,True,1,1
132013,1,0.84,0.84,0.84,0.84,0.84,0.84,0.84,-8.091814,-15.031449,False,False,3,1
132014,1,0.58,0.58,0.58,0.58,0.58,0.58,0.58,-8.091814,-14.936831,False,False,3,1
132015,1,0.01,0.01,0.01,0.01,0.01,0.01,0.01,-8.091814,-14.974067,False,False,3,1
